In [ ]:
import base64
import os
from datetime import datetime, timezone
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import padding
import csv
import blake3

# Function to generate a dynamic key based on the current minute
def generate_key():
    """Generate a dynamic key based on the current time."""
    current_time = datetime.now(timezone.utc)  # Get current time in UTC
    time_str = current_time.strftime("%Y%m%d%H%M")  # Get year, month, day, hour, and minute as a string
    key = base64.urlsafe_b64encode(time_str.encode('utf-8')).ljust(32, b'\0')
    return key[:32]

# Function to hash the key using blake3
def hash_key(key):
    """Hash the key using blake3."""
    return blake3.blake3(key).digest()

# Function to decrypt data using AES
def decrypt(encrypted_data, key):
    """Decrypt data using the provided key."""
    iv = encrypted_data[:16]
    encrypted_data = encrypted_data[16:]

    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()

    padded_data = decryptor.update(encrypted_data) + decryptor.finalize()

    unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
    data = unpadder.update(padded_data) + unpadder.finalize()
    
    return data

# Function to decrypt a CSV file
def decrypt_csv(encrypted_csv, decrypted_csv):
    """Decrypt a CSV file."""
    # Read the encrypted data from the file
    with open(encrypted_csv, 'rb') as infile:
        encrypted_data = infile.read()

    # Generate key based on current time
    key = generate_key()
    print(f"Generated Key (before hashing): {key}")

    # Hash the key using blake3
    hashed_key = hash_key(key)
    print(f"Hashed Key: {hashed_key}")

    # Decrypt the data
    data = decrypt(encrypted_data, hashed_key)

    # Convert bytes back to CSV rows
    rows = data.decode('utf-8').split("\n")
    rows = [row.split(",") for row in rows]

    # Save the decrypted data to a CSV file
    with open(decrypted_csv, 'w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(rows)

    print(f"Decrypted CSV saved to {decrypted_csv}")

# Example usage: Decrypt the CSV file
if __name__ == "__main__":
    encrypted_csv = "/Users/bodapati/Documents/encrypted_data.csv"  
    decrypted_csv = "/Users/bodapati/Documents/decrypted_data.csv"
    decrypt_csv(encrypted_csv, decrypted_csv)

Generated Key (before hashing): b'MjAyNTAzMTUxOTI2\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'
Hashed Key: b'\x11\xa1\xfc\x0e\xfe\xabw?qH\x97KY\xa3\xb8hU\xa7\xbb\xbcz\xaa(\xcbae\xd1T\xc8\xea\x0e\x1b'
Decrypted CSV saved to /Users/bodapati/Documents/decrypted_data.csv
